In [ ]:
import re

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

corpus = []

data = gc.open('train_data').sheet1.get_all_values()
data = [item for sublist in data for item in sublist]

for line in data:
  line = line.replace("。","").replace("、","")
  line = re.sub(r"\d+", "", line)
  corpus.append(line)

#informal = 0, polite = 1, formal = 2
def label_int(string_label):
  if string_label == "informal":
    return 0
  elif string_label == "polite":
    return 1
  else: 
    return 2

label = gc.open('train_label').sheet1.get_all_values()
label = [item for sublist in label for item in sublist]

y = list(map(lambda x: label_int(x), label))

In [ ]:
!pip install nagisa

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import nagisa
import pandas as pd

def tokenizer_jp(text):
  words = nagisa.filter(text, filter_postags=['助詞'])
  return words.words

# stop_words = ['な', 'と', 'た', 'で', 'は']

vectorizer = TfidfVectorizer(tokenizer=tokenizer_jp)

corpus_tfidf = vectorizer.fit_transform(corpus)

frame = pd.DataFrame(corpus_tfidf.toarray(), columns= vectorizer.get_feature_names())
print(frame)

X_train, X_test, y_train, y_test = train_test_split(corpus_tfidf, y, test_size = 0.2, random_state = 34)

      あっ  あなた   あの        あり  ありがとう   ある  ...   高く   高層   鳥肌   鶏肉    鷹  黄色く
0    0.0  0.0  0.0  0.345933    0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
1    0.0  0.0  0.0  0.000000    0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
2    0.0  0.0  0.0  0.000000    0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
3    0.0  0.0  0.0  0.000000    0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
4    0.0  0.0  0.0  0.000000    0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
..   ...  ...  ...       ...    ...  ...  ...  ...  ...  ...  ...  ...  ...
195  0.0  0.0  0.0  0.000000    0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
196  0.0  0.0  0.0  0.000000    0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
197  0.0  0.0  0.0  0.000000    0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
198  0.0  0.0  0.0  0.000000    0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0
199  0.0  0.0  0.0  0.000000    0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0

[200 rows x 885 columns]


In [ ]:
import pickle

pickle.dump(vectorizer, open('vectorizer.pkl', 'wb'))

## **Logistic Regression**

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver = 'liblinear', penalty = 'l2', C =10)

model.fit(X_train, y_train)

## **SGD Classifier**

In [ ]:
from sklearn.linear_model import SGDClassifier

model = SGDClassifier(loss='hinge', penalty='l2', max_iter=1000, alpha=0.0003, n_iter_no_change=10)

# model.fit(X_train, y_train)

## **Naive Bayes**

In [ ]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()

model.fit(X_train, y_train)

## **Support Vector Machine (SVM)**

In [ ]:
from sklearn.svm import SVC

model = SVC(C=1.0, kernel='linear', degree=3, gamma='auto')

model.fit(X_train, y_train)

## **Cross Validation**

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

cv = KFold(n_splits=10, random_state=1, shuffle=True)

model = 
scores = cross_val_score(model, corpus_tfidf, y, scoring='accuracy', cv=cv, n_jobs=-1)

print(scores)
a = 0
for i in scores:
  a += i

print(a/10)

# **train : test = 8 : 2**
100 sentences accuracy = 0.8

200 sentences accuracy = 0.85

In [ ]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report

y_pred = model.predict(X_test)

accuracy = metrics.accuracy_score(y_test, y_pred)

print(accuracy)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.925
[[14  1  0]
 [ 2 14  0]
 [ 0  0  9]]
              precision    recall  f1-score   support

           0       0.88      0.93      0.90        15
           1       0.93      0.88      0.90        16
           2       1.00      1.00      1.00         9

    accuracy                           0.93        40
   macro avg       0.94      0.94      0.94        40
weighted avg       0.93      0.93      0.93        40



In [ ]:
import pickle

pickle.dump(model, open('model.pkl', 'wb'))